# Adaptive RAG

PDF 테이블 정보에 대한 Recursive Retrieval 전략
- 다수의 CSV 테이블 대상으로 검색 chunk와 답변 생성 chunk 분리해보기
- 테이블 검색용 Chunk 대상으로 User Query에 대해 Adaptive하게 retrieval 가져가보기


In [ ]:
%pip install llama-index-embeddings-openai llama-index-llms-openai camelot-py llama-index pymupdf

In [ ]:
!apt-get install ghostscript
!pip install ghostscript

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ''

In [ ]:
import camelot

from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import PandasQueryEngine
from llama_index.core.schema import IndexNode
from llama_index.llms.openai import OpenAI

from llama_index.readers.file import PyMuPDFReader
from typing import List

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# 추후 사용할 llm, 임베딩 모델 클래스 정의
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [ ]:
# 파싱할 파일 경로 설정
file_path = "/content/billionaires_page.pdf"

In [ ]:
# PDF파서 정의
reader = PyMuPDFReader()

In [ ]:
# 업로드된 경로에서 로딩스테이지 진행한 후 다큐먼트 단위로 저장
docs =

In [ ]:
# 도큐먼트 정보 확인
docs

In [ ]:
from llama_index.core import Settings
#노드변환 및 파싱
doc_nodes =

In [ ]:
# 비교를 위한 Naive-RAG 구성
vector_index0 =
vector_query_engine0 =

In [ ]:

response = vector_query_engine0.query(
    ""
)

In [ ]:
print()

In [ ]:
print(str(response))

In [ ]:
response = vector_query_engine0.query(
    ""
)
print(str(response))

In [ ]:
print()

- 기본적인 PDF파싱모듈로는 테이블 등 Text-Only 가 아닌 문서에 대한 정보 해석력이 떨어지는 것을 확인
- Table정보를 따로 추출하여 답하는 방식은 어떨지?

In [ ]:
# pdf의 테이블파싱하기 - camelot 활용
def get_tables(path: str, pages: List[int]):
    table_dfs = []
    for page in pages:
        table_list = camelot.read_pdf(path, pages=str(page))
        for table in table_list:
            table_df = table.df
            table_df = (
                table_df.rename(columns=table_df.iloc[0])
                .drop(table_df.index[0])
                .reset_index(drop=True)
            )
            table_dfs.append(table_df)
    return table_dfs

In [ ]:
table_dfs = get_tables(file_path, pages=[])

In [ ]:
#파싱된 테이블 개수확인
len(table_dfs)

In [ ]:
#파싱 결과 확인


In [ ]:
#파싱 결과 확인


In [ ]:
#파싱 결과 확인


이제 테이블을 다 파싱해왔는데,
이것들을 기반으로 질문에 바로 답할수 있도록 만들려면 만들수도 있겠지만,
테이블이 지금과 다르게 수천 수만개일때, 모든 유저 쿼리에 대해 수만개의 테이블을 매번 조회하는 것은 실용성 없는 Naive한 접근방식(자원은 무한하지 않다).

그렇기 때문에,
1. 사용자의 질문과 관련된 테이블을 먼저 찾고
2. 찾은 테이블을 기준으로 사용자의 질문에 답할 수 있는 정보를 발췌하여 답해보자.

일단은 각 테이블별로 답해주는 담당 라마인덱스 쿼리엔진을 만들어주자.

(지난 주 커버한 쿼리엔진 라우터(AdaptiveRAG))

In [ ]:
!pip install llama-index-experimental

In [ ]:
from llama_index.experimental.query_engine import PandasQueryEngine

llm = OpenAI(model="gpt-4o-mini")

#table을 기준으로 하는 query engine(PandasQueryEngine) 각 추출 테이블별로 만들고 리스트화
df_query_engines = [

]

In [ ]:
df_query_engines

In [ ]:
# 상응하는 테이블 직접 지정해서 답변 요구
response =
print(str(response))

In [ ]:
# 상응하는 테이블 직접 지정해서 답변 요구
response =
print(str(response))

In [ ]:
table_dfs[0]

In [ ]:
# 상응하는 테이블 지정해서 답변 요구
response = df_query_engines[2].query(

)
print(str(response))

In [ ]:
# 상응하는 테이블 지정해서 답변 요구
response = df_query_engines[2].query(

)
print(str(response))

In [ ]:
table_dfs[2]

In [ ]:
response = df_query_engines[4].query(

)
print(str(response))

In [ ]:
response = df_query_engines[4].query(

)
print(str(response))

In [ ]:
table_dfs[4]

질문별로 담당하는 쿼리엔진을 부여하는 것으로 heuristic하게 서칭 스페이스를 줄이고 시작할 수 있는 것 확인

In [ ]:
# 쿼리엔진 요약문 생성
summaries = [
    (
        "This node provides information about the world's richest billionaires"
        " in 2023"
    ),
    (

    ),
    (

    ),
    (

    ),
    (
        "This node provides information on the number of billionaires and"
        " their combined net worth from 2000 to 2023."
    ),
]

#생성된 요약문 별 노드단위 생성
df_nodes = [
    IndexNode()
    for idx, summary in enumerate(summaries)
]

#요약노드 <-> 쿼리엔진 매핑
df_id_query_engine_mapping = {
    f"pandas{idx}":
    for idx, df_query_engine in enumerate(df_query_engines)
}

In [ ]:
#생성된 노드 확인
df_nodes[0]

In [ ]:
#상위레벨 벡터스토어인덱스 정의
vector_index =
vector_retriever =

RecursiveRetriever 활용한 Adaptive Engine Selection
- Pydantic Selector과 다르게 LLM 기반의 Function Calling이 아닌 2 step retrieval.
- chunk retrieve 하듯이 쿼리엔진별 description 대상으로 1차 거리계산 retrieve(top_k=1), 이후 retrieve 된 description을 가진 쿼리 엔진을 이용하여 쿼리 답안 생성

In [ ]:
from llama_index.core.retrievers import RecursiveRetriever

from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

In [ ]:
response = query_engine.query(

)

In [ ]:
# 하위 리트리버의 response


In [ ]:
# 그걸 전달받은 상위 리트리버의 최종 답안
str(response)

In [ ]:
response = query_engine.query(")

In [ ]:
str(response)

DO IT YOURSELF
- 손흥민 위키피디아 pdf 코랩 환경에 업로드
- 위 exercise와 마찬가지로, camelot으로 pdf 내 테이블 파싱
- 파싱된 테이블별 pandas query engine 구축
- query engine별 연관 질문 해보기
- recursive retrieval 활용하여 상위레벨 쿼리엔진 구조 완성 하여 손흥민 AdaptiveRAG 구현해보기

In [ ]:
# 파싱할 파일 경로 설정
file_path =

In [ ]:
# PDF파서 정의
reader =
# 업로드된 경로에서 로딩스테이지 진행한 후 다큐먼트 단위로 저장
docs =

In [ ]:
doc_nodes = Settings.node_parser.get_nodes_from_documents(docs)

In [ ]:
# 비교를 위한 Naive-RAG 구성
vector_index0 =
vector_query_engine0 =


In [ ]:
#질문1: 손흥민이 소속된 구단이 어디야?
response =

In [ ]:
print(str(response))

In [ ]:
print(response.source_nodes[0].node.get_content())

In [ ]:
# 질문2: 손흥민이 리그에서 23골을 넣었던 시즌이 언제야?
response =

In [ ]:
print(str(response))

In [ ]:
print(response.source_nodes[0].node.get_content())

In [ ]:
print(response.source_nodes[1].node.get_content())

In [ ]:
# 성능고도화를 위해 get_tables 함수로 camelot라이브러리 활용한 테이블 파싱(페이지 14, 15, 19)
table_dfs =

In [ ]:
len(table_dfs)

In [ ]:
#파싱 결과 확인
table_dfs[0]

In [ ]:
import pandas as pd
df = table_dfs[0].replace('\n', '', regex=True)

df['클럽'] = df['클럽'].replace('', pd.NA).fillna(method='ffill')
df['시즌'] = df['시즌'].replace('', pd.NA).fillna(method='ffill')

df.columns = [
    '클럽', '시즌', '리그', '리그_경기', '리그_골', '리그_도움',
    '국내컵_경기', '국내컵_골', '국내컵_도움', '리그컵_경기',
    '리그컵_골', '리그컵_도움', 'UEFA_경기', 'UEFA_골', 'UEFA_도움',
    '합계_경기', '합계_골', '합계_도움'
]

df = df.replace('—', pd.NA)

numeric_columns = [
    '리그_경기', '리그_골', '리그_도움',
    '국내컵_경기', '국내컵_골', '국내컵_도움',
    '리그컵_경기', '리그컵_골', '리그컵_도움',
    'UEFA_경기', 'UEFA_골', 'UEFA_도움',
    '합계_경기', '합계_골', '합계_도움'
]

df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
df = df.drop(index=0)

df = df.reset_index(drop=True)
table_dfs[0] = df

In [ ]:
table_dfs[0]

In [ ]:
table_dfs[1]

In [ ]:
table_dfs[2]

In [ ]:
# 파싱된 DF별 판다스쿼리엔진 assign
df_query_engines =

In [ ]:
# 질문: "손흥민이 리그에서 23골을 넣었던 시즌이 언제야?"
response =
print(str(response))

In [ ]:
# 질문: "손흥민이 상대팀 리버풀 상대로 몇골 넣었지?"
response =
print(str(response))

In [ ]:
# 질문: "손흥민이 어떤 팀 상대로 가장 많은 골을 넣었지?"
response =
print(str(response))

In [ ]:
# 질문: "2018 FIFA 월드컵에서 어떤 팀들 상대로 골 넣었었지?"
response =
print(str(response))

In [ ]:
table_dfs[0]

In [ ]:
# 쿼리엔진 요약문 생성
summaries = [
    (
        "<이 쿼리엔진이 타게팅될 수 있도록 설명 명확히 작성해보기>"
    ),
    (
        "<이 쿼리엔진이 타게팅될 수 있도록 설명 명확히 작성해보기>"
    ),
    (
        "<이 쿼리엔진이 타게팅될 수 있도록 설명 명확히 작성해보기>"
    ),
]

#생성된 요약문 별 노드단위 생성
df_nodes = [
    IndexNode(text=summary, index_id=f"pandas{idx}")
    for idx, summary in enumerate(summaries)
]

#요약노드 <-> 쿼리엔진 매핑
df_id_query_engine_mapping = {
    f"pandas{idx}": df_query_engine
    for idx, df_query_engine in enumerate(df_query_engines)
}

In [ ]:
#상위레벨 벡터스토어인덱스 정의
vector_index =
vector_retriever =

In [ ]:
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=df_id_query_engine_mapping,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

In [ ]:
#질문: "손흥민이 상대팀 리버풀 상대로 몇골 넣었지?"
response =
print(str(response))

In [ ]:
#질문: "손흥민이 리그에서 23골을 넣었던 시즌이 언제야?"
response =
print(str(response))

In [ ]:
#질문: "손흥민이 어떤 팀 상대로 가장 많은 골을 넣었지?"
response =
print(str(response))


In [ ]:
#질문: "2018 FIFA 월드컵에서 어떤 팀들 상대로 골 넣었었지?"
response =
print(str(response))
